In [1]:
import os
import cv2
import numpy as np

In [2]:
def getNextNotAnnotatedImage(pathToAnnotate):
    '''Goes through the directory pathToAnnotate, looks for the first image that has no corresponding txt file, 
    i.e. not annotated image, and returns its path'''
    
    listImagesToAnnotate = os.listdir(pathToAnnotate)
    for imageName in listImagesToAnnotate:
        if '.jpg' in imageName:
            imageNameNoExtension = imageName.replace('.jpg','')
            if imageNameNoExtension+'.csv' not in listImagesToAnnotate:
                return pathToAnnotate+imageName
            
    return -1

In [3]:
def click(event, x, y, flags, param):
    # grab references to the global variables
    global refPt, cropping

    # if the left mouse button was clicked, record the
    # (x, y) coordinates
    if event == cv2.EVENT_LBUTTONDOWN:
        pt = (x, y)
        gravePositions.append(pt)

        # draw a point where the user has clicked
        cv2.circle(clone, pt, 1, (0, 255, 0), 7)
        cv2.imshow("image", clone)

# Cell that loads the next image to annotate

In [4]:
pathToAnnotate = '../imagesToAnnotate/'
imagePath= getNextNotAnnotatedImage(pathToAnnotate)
if imagePath == -1:
    print('All of the images in the directory ../imagesToAnnotate are annotated')
else:
    image = cv2.imread(imagePath)
    imageShape = image.shape
    
    #Resizing the image to make it fit the screen, you may have to change these values depending on your screen resolution
    resizingShape = (1200,600)
    #######################################
    
    imageResized = cv2.resize(image, resizingShape)

    #create a copy of the image
    clone = imageResized.copy()

    # initialize the list of grave positions
    gravePositions = []

    cv2.namedWindow("image")
    cv2.setMouseCallback("image", click)

    # keep looping until the 'q' key is pressed
    while True:
        # display the image and wait for a keypress
        cv2.imshow("image", clone)
        key = cv2.waitKey(1) & 0xFF

        # if the 'p' key is pressed, remove the last clicked point from the list
        if key == ord('p'):
            gravePositions = gravePositions[:-1]
            clone = imageResized.copy()
            for pt in gravePositions:
                cv2.circle(clone, pt, 1, (0, 255, 0), 7)
            cv2.imshow("image", clone)


        # if the 's' key is pressed, break from the loop and SAVE the grave positions
        if key == ord("s"):
            imagePathNoExtension = imagePath.replace('.jpg','')
            
            #Map the grave positions in the resizing shape to the grave positions in the original shape of the image
            height, width, _ = imageShape
            resizingWidth, resizingHeight = resizingShape
            gravePositionsOriginal = [(float(x)/resizingWidth*width,float(y)/resizingHeight*height) for x,y in gravePositions]
            
            np.savetxt(imagePathNoExtension+'.csv', np.array(gravePositionsOriginal))
            break

        # if the 'q' key is pressed, break from the loop, DOEST'T SAVE the positions
        elif key == ord("q"):
            break

    # close all open windows
    cv2.destroyAllWindows()

# How to get the grave positions later

In [139]:
imageAnnotated = image.copy()
gravePositionsOriginal = np.loadtxt(imagePathNoExtension+'.csv')
gravePositionsOriginal = np.array(gravePositionsOriginal).reshape((-1,2))
for i in range(len(list(gravePositionsOriginal))):
    pt = tuple(gravePositionsOriginal[i,:])
    pt = (int(round(pt[0])), int(round(pt[1])))
    cv2.circle(imageAnnotated, pt, 1, (0, 255, 0), 7)
    
imageAnnotatedResized = cv2.resize(imageAnnotated, resizingShape)
cv2.imshow('Annotated image', imageAnnotatedResized)
cv2.waitKey()    

-1